ResNet-50

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report


In [ ]:
def create_resnet_model():

    # Load the pre-trained ResNet50 model without the top layer
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Freeze the weights of the pre-trained layers
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Add the final classification layer with sigmoid activation for binary classification
    predictions = Dense(1, activation='sigmoid')(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)
    return model


In [ ]:
# Create the ResNet-based model
resnet_model = create_resnet_model()

# Compile the model using RMSprop optimizer, binary crossentropy loss, and accuracy metric
resnet_model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Define a checkpoint callback
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')

# Train the model for 10 epochs with a batch size of 32 and a validation split of 0.1
resnet_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, callbacks=[checkpoint])


In [ ]:
# Evaluate the model on the test set
resnet_model.evaluate(X_test, y_test)


In [ ]:
# Load the best model from the checkpoint
resnet_model.load_weights("best_model.h5")

# Make predictions on the test set
y_pred_resnet = resnet_model.predict(X_test)

# Convert the predictions to binary labels (0 or 1) using a threshold of 0.5
y_pred_resnet = (y_pred_resnet > 0.5).astype(int)


In [ ]:
print(classification_report(y_test, y_pred_resnet))

CNN (adam optimizer):

In [ ]:
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


In [ ]:
real_dir = "/kaggle/input/celeb-real"
fake_dir = "/kaggle/input/celeb-synthesis"


In [ ]:
def extract_frames(video_dir, image_dir):
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)
    for video in os.listdir(video_dir):
        cap = cv2.VideoCapture(os.path.join(video_dir, video))
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_index = np.random.randint(0, frame_count)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        success, frame = cap.read()
        if success:
            frame = cv2.resize(frame, (224, 224))
            video_name = os.path.splitext(video)[0]
            image_name = video_name + "_" + str(frame_index) + ".jpg"
            cv2.imwrite(os.path.join(image_dir, image_name), frame)
        cap.release()


In [ ]:
extract_frames(real_dir, "/kaggle/working/real_images")
extract_frames(fake_dir, "/kaggle/working/fake_images")


In [ ]:
def load_data(image_dirs, labels):
    images = []
    image_labels = []
    for image_dir, label in zip(image_dirs, labels):
        for image in os.listdir(image_dir):
            img = cv2.imread(os.path.join(image_dir, image))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img / 255.0
            images.append(img)
            image_labels.append(label)
    images = np.array(images)
    image_labels = np.array(image_labels)
    return images, image_labels


In [ ]:
X, y = load_data(["/kaggle/working/real_images", "/kaggle/working/fake_images"], [0, 1])


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation="sigmoid"))
    return model


In [ ]:
model = create_model()
model.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


In [ ]:
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_loss", mode="min")
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, callbacks=[checkpoint])


In [ ]:
model.evaluate(X_test, y_test)


In [ ]:
model.load_weights("best_model.h5")
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)


In [ ]:
print(classification_report(y_test, y_pred))


XceptionNet

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from PIL import Image


In [ ]:
# Define data transformations for training and validation
train_transforms = transforms.Compose([
    transforms.Resize((299, 299)), # Resize for Xception input size
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

val_transforms = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load custom dataset
train_data = datasets.ImageFolder('path_to_training_data', transform=train_transforms)
val_data = datasets.ImageFolder('path_to_validation_data', transform=val_transforms)

# Define data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=32)


In [ ]:
# Load pre-trained Xception model
model = torch.hub.load('pytorch/vision:v0.10.0', 'xception', pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2) # Assuming 2 classes: Real and Fake

# Freeze initial layers and only train the last few layers
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True


In [ ]:
# Define loss function and optimizer with weight decay
criterion = nn.CrossEntropyLoss()
weight_decay = 1e-4
optimizer = optim.Adam(model.fc.parameters(), lr=0.001, weight_decay=weight_decay)

# Define learning rate scheduler
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)


In [ ]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [ ]:
# Training loop with learning rate scheduling
num_epochs = 10
best_accuracy = 0.0
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    # Update learning rate
    scheduler.step()

    # Validation
    model.eval()
    correct_predictions = 0
    total_predictions = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    epoch_accuracy = correct_predictions / total_predictions
    if epoch_accuracy > best_accuracy:
        best_accuracy = epoch_accuracy
        torch.save(model.state_dict(), "best_xception_model.pth") # Save the best model

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_data)}, Accuracy: {epoch_accuracy}")
print(f"Best Validation Accuracy: {best_accuracy}")


In [ ]:
import torch
import torch.nn.functional as F
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import os

model_name = 'Wvolf/ViT_Deepfake_Detection'
model = ViTForImageClassification.from_pretrained(model_name)

# Define transformations to preprocess the uploaded image
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
transform = lambda img: feature_extractor(img, return_tensors='pt')

# Function to process an image and get the predicted label
def get_predicted_label(image_path):
    uploaded_image = Image.open(image_path)
    processed_image = transform(uploaded_image)['pixel_values']

    model.eval() # Set the model to evaluation mode
    with torch.no_grad():
        outputs = model(processed_image)
        predictions = F.softmax(outputs.logits, dim=1)
        predicted_class = torch.argmax(predictions, dim=1).item()

    # Decode the predicted class (0 for Real, 1 for Fake)
    class_labels = ['Real', 'Fake']
    predicted_label = class_labels[predicted_class]
    return predicted_label


In [ ]:
# Define the paths to the real and fake faces directories
real_faces_dir = "/content/drive/MyDrive/real_faces/real_faces/DeepFake00/DeepFake00"
fake_faces_dir = "/content/drive/MyDrive/fake_faces/fake_faces/DeepFake02/DeepFake02"

# Get the list of image files in each directory
real_images = [os.path.join(real_faces_dir, file) for file in os.listdir(real_faces_dir) if file.endswith('.jpg')]
fake_images = [os.path.join(fake_faces_dir, file) for file in os.listdir(fake_faces_dir) if file.endswith('.jpg')]

# Calculate accuracy for real faces
total_real_images = len(real_images)
if total_real_images > 0:
    correct_predictions_real = 0

    for image_path in real_images:
        predicted_label = get_predicted_label(image_path)

        if predicted_label == 'Real':
            correct_predictions_real += 1

    accuracy_real = correct_predictions_real / total_real_images
else:
    accuracy_real = 0.0 # Set accuracy to 0 if there are no real images

# Calculate accuracy for fake faces
total_fake_images = len(fake_images)
if total_fake_images > 0:
    correct_predictions_fake = 0

    for image_path in fake_images:
        predicted_label = get_predicted_label(image_path)

        if predicted_label == 'Fake':
            correct_predictions_fake += 1

    accuracy_fake = correct_predictions_fake / total_fake_images
else:
    accuracy_fake = 0.0 # Set accuracy to 0 if there are no fake images

# Calculate accuracy for all faces (real and fake)
total_images = total_real_images + total_fake_images
if total_images > 0:
    correct_predictions_all = correct_predictions_real + correct_predictions_fake
    accuracy_all = correct_predictions_all / total_images
else:
    accuracy_all = 0.0 # Set accuracy to 0 if there are no images

print(f'Overall Accuracy: {accuracy_all * 100:.2f}%')
